In [1]:
from guidance import models, system, user, assistant, gen, select
from datasets import load_dataset

/home/daniel/private_code/llmtopic/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d = load_dataset("yelp_polarity")

In [3]:
dt = d["train"]

In [13]:
llm = models.LlamaCppChat("../models/dolphin-2_6-phi-2.Q4_K_M.gguf", n_gpu_layers=-1, n_threads=8, n_threads_batch=8, n_ctx=2048, chat_format="chatml", echo=False)

In [14]:
with system():
    llm += "You are a helpful assistant."

In [17]:
from tqdm import tqdm

prompt_template1 = (
    "Please analyze the following document and determine the key topics to extract, "
    "ranging from 0 to {max_topics}. The topics should be directly derived from the document's content. "
    "Each extracted topic should be no more than three words long and must reflect the main points or themes present in the document.\n\n"
    "Criteria for topics: {custom_criteria}\n\n"
    "After analyzing the document, present the topics in a numbered list format. Each topic should be a concise representation of a specific theme or point from the document. "
    "Avoid using generic placeholders. Instead, use actual terms or phrases found in the text. For example:\n"
    "1. Customer Feedback\n"
    "2. Service Quality\n"
    "Each topic should start on a new line.\n\n"
    "Document:\n{document}\n\n"
)

custom_criteria = (
    "Focus on the main topics relevant to customer satisfaction or dissatisfaction, being as specific as possible and using actual terms from the document."
)

max_topics = 5  # Set the maximum number of topics you want to allow

# Rest of your code follows...


prompt_template2 = (
    "Extracted Topics:\n"
)

custom_criteria = (
    "Please focus on the main topics relevant to customer satisfaction or dissatisfaction. Be as specific as possible when describing the topics."
)

max_topics = 5

all_topics = []

for sample in tqdm(dt, desc="Processing samples", unit="sample"):
    tmp_llm = llm
    with user():
        tmp_llm += prompt_template1.format(max_topics=max_topics, document=sample["text"], custom_criteria=custom_criteria)

    with assistant():
        tmp_llm += "Number of Topics to Extract: "+  select(list(range(max_topics+1)), name="num_topics")

    num_topics = int(tmp_llm["num_topics"])
    
    with assistant():
        for i in range(num_topics):
            tmp_llm += "Extracted topics:\n" + f"{i+1}. " + gen(name="topic", list_append=True, max_tokens=10, stop="\n", suffix="\n", regex=r"[a-zA-Z\s]+")
    
    
    all_topics.append(tmp_llm["topic"])

Processing samples:   0%|                                                                                                         | 51/560000 [00:23<71:47:56,  2.17sample/s]


KeyboardInterrupt: 

In [18]:
all_topics

[['Customer Feedback', 'Service Quality'],
 ['Customer Feedback',
  'Service Quality',
  'Medical Expertise',
  'Patient Experience',
  'Personalized Care'],
 ['Customer Feedback',
  'Service Quality',
  'Medical Care',
  'Office Staff',
  'Financial Concerns'],
 ['Unprofessional office staff',
  'Unanswered messages',
  'Unnecessary blood work',
  'Preventative visits',
  'Insurance coverage issues'],
 ['Customer Feedback', 'Service Quality', 'Food', 'Atmosphere', 'Price'],
 ['Customer Feedback', 'Service Quality', 'Sauce', 'Wings', 'Fish Sandwich'],
 ['Customer Feedback',
  'Service Quality',
  'Accessible Hours',
  'Pit to Hit Into',
  'Cash Only Policy'],
 ['Customer Feedback',
  'Service Quality',
  'Availability of Tees',
  'Vending Machines',
  'Batting Cage'],
 ['Customer Feedback',
  'Service Quality',
  'Range Size',
  'Pro Shop Friendliness',
  'Extra Charges'],
 ['Customer Feedback',
  'Service Quality',
  'New Range',
  'Club House',
  'Golf Course Experience'],
 ['Custome